In [ ]:
import pandas as pd
import numpy as np
from functools import reduce
import pyodbc
from sqlalchemy import create_engine
import urllib

In [ ]:
# ar = pd.read_csv('../ar_reports/orig/legion_of_bloom_detail.csv')
# ar = pd.read_csv('../ar_reports/orig/biscotti_ar.csv')
# ar = pd.read_csv('../ar_reports/orig/elefante_ar.csv')
# ar = pd.read_csv('../ar_reports/orig/dna_organics_ar.csv')
# ar = pd.read_csv('../ar_reports/orig/jc_rad_ar.csv')
# ar = pd.read_csv('../ar_reports/orig/mgvs_ar.csv')
# ar = pd.read_csv('../ar_reports/orig/crown_genetics_ar.csv')
# ar = pd.read_csv('../ar_reports/orig/lifted_ar.csv')
# ar = pd.read_csv('../ar_reports/orig/herer_ar.csv')
# ar = pd.read_csv('../ar_reports/orig/sd_strains_ar.csv')

In [ ]:
file = "bear_extraction_ar"

In [ ]:
ar = pd.read_csv(f'../ar_reports/orig/{file}.csv')

In [ ]:
ar.head(2)

In [ ]:
ar.info()

In [ ]:
# ar_null_license = ar[ar['state_license'].isnull()]
# ar_full_license = ar[~ar['state_license'].isnull()]

In [ ]:
ar_final = ar[ar['license_number'] != '0']

In [ ]:
ar_final

In [ ]:
ar_final[ar_final['days_over_due'] <0]

In [ ]:
for i in range(len(ar_final)):
    if ar_final['days_over_due'].iloc[i] < 0:
        ar_final['days_over_due'].iloc[i] = 0
    if ar_final['orig_amt'].iloc[i] < 0:
        ar_final['orig_amt'].iloc[i] = 0
    if ar_final['current'].iloc[i] < 0:
        ar_final['current'].iloc[i] = 0
    if ar_final['30_days'].iloc[i] < 0:
        ar_final['30_days'].iloc[i] = 0
    if ar_final['60_days'].iloc[i] < 0:
        ar_final['60_days'].iloc[i] = 0
    if ar_final['90_days'].iloc[i] < 0:
        ar_final['90_days'].iloc[i] = 0
    if ar_final['over_90_days'].iloc[i] < 0:
        ar_final['over_90_days'].iloc[i] = 0

In [ ]:
count_payor = ar_final.groupby(['ar_account', 'date'], as_index=False)['license_number'].count()

In [ ]:
# count_payor = ar_final.groupby('ar_account', as_index=False)['license_number'].count()

In [ ]:
count_payor.head()

In [ ]:
count_payor.rename(columns={'license_number': 'count_payor'}, inplace=True)

In [ ]:
sum_days_over = ar_final.groupby(['ar_account', 'date'], as_index=False)['days_over_due'].sum()
sum_orig_amt_clean = ar_final.groupby(['ar_account', 'date'], as_index=False)['orig_amt'].sum()
sum_bal_out_clean = ar_final.groupby(['ar_account', 'date'], as_index=False)['bal_out'].sum()
sum_current_clean = ar_final.groupby(['ar_account', 'date'], as_index=False)['current'].sum()
sum_30_day_clean = ar_final.groupby(['ar_account', 'date'], as_index=False)['30_days'].sum()
sum_60_day_clean = ar_final.groupby(['ar_account', 'date'], as_index=False)['60_days'].sum()
sum_90_day_clean = ar_final.groupby(['ar_account', 'date'], as_index=False)['90_days'].sum()
sum_over_90_day_clean = ar_final.groupby(['ar_account', 'date'], as_index=False)['over_90_days'].sum()

In [ ]:
# sum_delinquent_clean = ar_final.groupby(['ar_account', 'date'], as_index=False)['delinquent'].max()

In [ ]:
# sum_delinquent_clean

In [ ]:
# sum_days_over = ar_final.groupby('ar_account', as_index=False)['days_over_due'].sum()
# sum_orig_amt_clean = ar_final.groupby('ar_account', as_index=False)['orig_amt'].sum()
# sum_bal_out_clean = ar_final.groupby('ar_account', as_index=False)['bal_out'].sum()
# sum_current_clean = ar_final.groupby('ar_account', as_index=False)['current'].sum()
# sum_30_day_clean = ar_final.groupby('ar_account', as_index=False)['30_days'].sum()
# sum_60_day_clean = ar_final.groupby('ar_account', as_index=False)['60_days'].sum()
# sum_90_day_clean = ar_final.groupby('ar_account', as_index=False)['90_days'].sum()
# sum_over_90_day_clean = ar_final.groupby('ar_account', as_index=False)['over_90_days'].sum()

In [ ]:
data_frames = [count_payor, sum_days_over, sum_orig_amt_clean, sum_bal_out_clean, sum_current_clean, sum_30_day_clean,
               sum_60_day_clean, sum_90_day_clean, sum_over_90_day_clean]
#               sum_delinquent_clean]

In [ ]:
df_sums = reduce(lambda left,right: pd.merge(left,right,on=['ar_account', 'date'], how='outer'), data_frames)

In [ ]:
# df_sums = reduce(lambda left,right: pd.merge(left,right,on=['ar_account'], how='outer'), data_frames)

In [ ]:
df_sums

In [ ]:
ar_to_merge = ar_final[['license_number', 'ar_account']]

In [ ]:
len(ar_to_merge)

In [ ]:
ar_to_merge

drop duplicates in the list that has the license numbers associated with all ar accounts from file

In [ ]:
pd.set_option('mode.chained_assignment', None)

ar_to_merge.drop_duplicates(subset='ar_account', keep='first', inplace=True)

In [ ]:
df_sums = pd.merge(df_sums, ar_to_merge, on='ar_account', how='outer')

In [ ]:
df_sums

In [ ]:
list2 = df_sums[df_sums['bal_out'] < 0].index
len(list2)

In [ ]:
pd.set_option('mode.chained_assignment', None)

df_sums['orig_amt'].loc[list2] = 0
df_sums['bal_out'].loc[list2] = 0
df_sums['current'].loc[list2] = 0
df_sums['30_days'].loc[list2] = 0
df_sums['60_days'].loc[list2] = 0
df_sums['90_days'].loc[list2] = 0
df_sums['over_90_days'].loc[list2] = 0

In [ ]:
for x in range(len(df_sums)):
    if df_sums['over_90_days'].iloc[x] < 0:
        val1 = df_sums['over_90_days'].iloc[x] + df_sums['90_days'].iloc[x]
        if val1 < 0:
            val2 = df_sums['60_days'].iloc[x] + val1
            if val2 < 0:
                val3 = df_sums['30_days'].iloc[x] + val2
                if val3 < 0:
                    val4 = df_sums['current'].iloc[x] + val3
                    df_sums['over_90_days'].iloc[x] = 0
                    df_sums['90_days'].iloc[x] = 0
                    df_sums['60_days'].iloc[x] = 0
                    df_sums['30_days'].iloc[x] = 0
                    df_sums['current'].iloc[x] = val4
                else:
                    df_sums['over_90_days'].iloc[x] = 0
                    df_sums['90_days'].iloc[x] = 0
                    df_sums['60_days'].iloc[x] = 0
                    df_sums['30_days'].iloc[x] = val3
            else:
                df_sums['over_90_days'].iloc[x] = 0
                df_sums['90_days'].iloc[x] = 0
                df_sums['60_days'].iloc[x] = val2
        else:
            df_sums['over_90_days'].iloc[x] = 0
            df_sums['90_days'].iloc[x] = val1
                    

In [ ]:
for x in range(len(df_sums)):
    if df_sums['90_days'].iloc[x] < 0:
        val1 = df_sums['90_days'].iloc[x] + df_sums['60_days'].iloc[x]
        if val1 < 0:
            val2 = df_sums['30_days'].iloc[x] + val1
            if val2 < 0:
                val3 = df_sums['current'].iloc[x] + val2
                df_sums['90_days'].iloc[x] = 0
                df_sums['60_days'].iloc[x] = 0
                df_sums['30_days'].iloc[x] = 0
                df_sums['current'].iloc[x] = val3
            else:
                df_sums['90_days'].iloc[x] = 0
                df_sums['60_days'].iloc[x] = 0
                df_sums['30_days'].iloc[x] = val2
        else:
            df_sums['90_days'].iloc[x] = 0
            df_sums['60_days'].iloc[x] = val1
          

In [ ]:
for x in range(len(df_sums)):
    if df_sums['60_days'].iloc[x] < 0:
        val1 = df_sums['60_days'].iloc[x] + df_sums['30_days'].iloc[x]
        if val1 < 0:
            val2 = df_sums['current'].iloc[x] + val1
            df_sums['60_days'].iloc[x] = 0
            df_sums['30_days'].iloc[x] = 0
            df_sums['current'].iloc[x] = val2
        else:
            df_sums['60_days'].iloc[x] = 0
            df_sums['30_days'].iloc[x] = val1

In [ ]:
for x in range(len(df_sums)):
    if df_sums['30_days'].iloc[x] < 0:
        val1 = df_sums['30_days'].iloc[x] + df_sums['current'].iloc[x]
        df_sums['30_days'].iloc[x] = 0
        df_sums['current'].iloc[x] = val1

In [ ]:
for x in range(len(df_sums)):
    if df_sums['current'].iloc[x] < 0:
        val1 = df_sums['current'].iloc[x] + df_sums['30_days'].iloc[x]
        if val1 < 0:
            val2 = df_sums['60_days'].iloc[x] + val1
            if val2 < 0:
                val3 = df_sums['90_days'].iloc[x] + val2
                if val3 < 0:
                    val4 = df_sums['over_90_days'].iloc[x] + val3
                    df_sums['current'].iloc[x] = 0
                    df_sums['30_days'].iloc[x] = 0
                    df_sums['60_days'].iloc[x] = 0
                    df_sums['90_days'].iloc[x] = 0
                    df_sums['over_90_days'].iloc[x] = val4
                else:
                    df_sums['current'].iloc[x] = 0
                    df_sums['30_days'].iloc[x] = 0
                    df_sums['60_days'].iloc[x] = 0
                    df_sums['over_90_days'].iloc[x] = val3
            else:
                df_sums['current'].iloc[x] = 0
                df_sums['30_days'].iloc[x] = 0
                df_sums['60_days'].iloc[x] = val2
        else:
            df_sums['current'].iloc[x] = 0
            df_sums['30_days'].iloc[x] = val1
                    

In [ ]:
df_sums.head(1)

In [ ]:
list1 = df_sums.iloc[:,5:9].apply(lambda x: pd.Series([(x < 0).sum()]), axis=1)
max(list1)

In [ ]:
df_final = df_sums

In [ ]:
df_final

In [ ]:
# df_final['date'] = pd.datetime(2019, 6, 30)
# df_final['client'] = 'Kiva'

In [ ]:
df_final['summary'] = 0

In [ ]:
df_final['license_provided'] = 0

In [ ]:
df_final

In [ ]:
df_final.rename(columns={'days_over_due': 'sum_days_over'}, inplace=True)

In [ ]:
df_final['delinquent'] = 0

In [ ]:
df_final = df_final[['license_number', 
          'license_provided',
          'ar_account', 
          'count_payor', 
          'sum_days_over',
           'orig_amt', 
          'bal_out', 
          'current', 
          '30_days', 
          '60_days', 
          '90_days',
           'over_90_days', 
          'delinquent', 
          'date', 
          'client', 
          'summary'
       ]]

In [ ]:
df_final.to_csv(f'../ar_reports/orig/no_neg/{file}.csv', index=False)

In [ ]:
# df_delinq = pd.read_csv('../ar_reports/orig/no_neg/LOB_delinq.csv')
# df_delinq.head(1)
# df_final['delinquent'] = df_final['ar_account'].isin(df_delinq['ar_account']).astype(int)
# len(df_final[df_final['delinquent'] == 1])

In [ ]:
# ar.drop(columns='Unnamed: 15', inplace=True)

In [ ]:
# df_sums.rename(columns={'state_license': 'license_number'}, inplace=True)aaaa

In [ ]:
# ar_full_license = ar[~ar['state_license'].isnull()]
# ar_missing = ar_null_license['ar_account'].unique()
# num_vals = len(ar_missing)
# array_list = []
# max_val = 0
# for i in range(num_vals):
#     array_list.append("assign_" + str(max_val))
#     max_val += 1
# id_array = np.asarray(array_list)
# pd_miss_fill = pd.DataFrame({'ar_account': ar_missing, 'state_license': id_array})
# ar_null_pd = ar_null_license.drop(columns='state_license')
# ar_fill = ar_null_pd.merge(pd_miss_fill, on='ar_account', how='left')
# ar_final = ar_full_license.append(ar_fill, sort=False, ignore_index=True)